In [15]:
import pandas as pd
import numpy as np

Make a function to calculate the average availability per location, and generate a new data frame with the results 

In [47]:
# NOTE: due to the use of both Macbooks and Windows computers in the project, the file path uses either / or \ in the file path
# as such, we have included a boolean to change the file path such that it works for the user's computer

macbook = False

import pandas as pd 

# now make a data frame with the average number of rentals per month per location. 
# In order to do this, have to use the same code as above for each city 

# Make a function such that it's easier to use for all locations
def avg_availability_location(location_name):
    # read the file for the location (for now, use Amsterdam)
    if macbook == False:
        location_path = "home_rentals" + location_name + "\calendar.csv.gz"
    if macbook == True:
        location_path = "home_rentals" + location_name + "/calendar.csv.gz"

    calendar = pd.read_csv(location_path)

    # make a new column which only has month-year 
    calendar["date"] = pd.to_datetime(calendar["date"])
    calendar["month"] = calendar["date"].dt.month
    calendar["year"] = calendar["date"].dt.year
    calendar['month_year'] = calendar['month'].astype(str) + '-' + calendar['year'].astype(str)

    # make the new data frame
    available_by_month = (calendar[calendar['available'] == 't'].groupby(['listing_id', 'month_year']).size().unstack(fill_value=0).stack().reset_index(name='days_available'))

    # Calculate the average days_available for each month_year (making sure that it rounds to full numbers)
    average_by_month = (available_by_month.groupby('month_year')['days_available'].mean().round(0).reset_index(name='average_days_available'))

    # convert the month_year column to datetime format 
    average_by_month['month_year'] = pd.to_datetime(average_by_month['month_year'], format='%m-%Y').dt.to_period('M')

    # sort the rows by ascending dates
    average_by_month = average_by_month.sort_values('month_year')

    # reset the row index
    average_by_month.reset_index(drop=True, inplace=True)

    # add the name of the location to the data frame 
    average_by_month["location"] = location_name[1:]

    return average_by_month



# run the function for each location, appending the location's df to an empty one for all of the locations

all_locations = [r"\Amsterdam", r"\Barcelona", r"\Euskadi", r"\Geneva", r"\Madrid", r"\Mallorca", r"\Malta", r"\Menorca", r"\Mexico City", r"\Milan", r"\Naples", 
                 r"\Rotterdam", r"\San Diego", r"\San Francisco", r"\Tokyo", r"\Toronto", r"\Valencia", r"\Vancouver", r"\Vienna", r"\Washington", r"\Winnipeg", r"\Zurich"]

counter = 0
if macbook == True:
    for location in all_locations:
        # Convert the string to a list of characters
        str_location = list(location)
        str_location[0] = "/"
        # Join the list back into a string
        location = ''.join(str_location)
        all_locations[counter] = location
        counter +=1

available_all_loc = pd.DataFrame()

# Create a list to store DataFrames for each location
dfs_list = []

for location in all_locations:
    average_by_month = avg_availability_location(location)
    dfs_list.append(average_by_month)


# Concatenate all DataFrames in the list
available_all_loc = pd.concat(dfs_list, ignore_index=True)

# visualise the final data frame
available_all_loc




,month_year,average_days_available,location
0,2022-09,4.0,Amsterdam
1,2022-10,8.0,Amsterdam
2,2022-11,12.0,Amsterdam
3,2022-12,12.0,Amsterdam
4,2023-01,14.0,Amsterdam
...,...,...,...
282,2023-05,17.0,Zurich
283,2023-06,16.0,Zurich
284,2023-07,16.0,Zurich
285,2023-08,17.0,Zurich


In [59]:
# check that each location has data for the same months

all_locations = ["Amsterdam", "Barcelona", "Euskadi", "Geneva", "Madrid", "Mallorca", "Malta", "Menorca", "Mexico City", "Milan", "Naples", 
                 "Rotterdam", "San Diego", "San Francisco", "Tokyo", "Toronto", "Valencia", "Vancouver", "Vienna", "Washington", "Winnipeg", "Zurich"]

for location in all_locations:
    one_loc = available_all_loc[available_all_loc["location"] == location]
    all_dates = one_loc["month_year"].unique()
    print(location, all_dates)




Amsterdam <PeriodArray>
['2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
 '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09']
Length: 13, dtype: period[M]
Barcelona <PeriodArray>
['2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
 '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09']
Length: 13, dtype: period[M]
Euskadi <PeriodArray>
['2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
 '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10']
Length: 14, dtype: period[M]
Geneva <PeriodArray>
['2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
 '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09']
Length: 13, dtype: period[M]
Madrid <PeriodArray>
['2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
 '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09']
Length: 13, dtype: period[M]
Mallorc

As can be seen above, all of the locations have data ranging from 2022-09 to 2023-09, except for Euskadi which has an extra month of data (2023-10). This is an additional month in comparison to the other locations, as such we will remove it to have a consistent range of data between all of the locations. In addition, we can assume that the data for the 2023-10 is incorrect since it predicts there the month has already been fully booked, but when looking at the spread of data for the previous months (almost all of them have at least half of the month unbooked) this seems highly unlikely. 

In [66]:
# remove the additional month of data for Euskadi

# Create a boolean mask based on the specified conditions
mask = available_all_loc['month_year'] == "2023-10"

# Use the boolean mask to filter the DataFrame and keep only the rows that don't match the conditions
available_all_loc = available_all_loc[~mask]

# check that we got the result we wanted
available_all_loc[available_all_loc["location"]=="Euskadi"]

,month_year,average_days_available,location
26,2022-09,2.0,Euskadi
27,2022-10,17.0,Euskadi
28,2022-11,20.0,Euskadi
29,2022-12,21.0,Euskadi
30,2023-01,19.0,Euskadi
31,2023-02,18.0,Euskadi
32,2023-03,19.0,Euskadi
33,2023-04,15.0,Euskadi
34,2023-05,16.0,Euskadi
35,2023-06,16.0,Euskadi


We  have no successfully generated a data frame with all of the data required to build our model.

In [67]:
''' 

TO DO:
- do some type of check for outliers and whatnot to show that all of our data is valid (although tbh idk if this is necessary, maybe we can 
    do a check that there is no data where the number of available days > number of days in the month, and also maybe check if there are any
    locations with very strange distributions of availabilities over the months??)
- split the data set into test and train 
- check when the data starts and ends --> if we don't have full months then it won't be valid (e.g. if it starts towards the end of september
    then it's skewed data because it'll say it was either completely free/completely booked for that month)


'''

" \n\nTO DO:\n- do some type of check for outliers and whatnot to show that all of our data is valid (although tbh idk if this is necessary, maybe we can \n    do a check that there is no data where the number of available days > number of days in the month, and also maybe check if there are any\n    locations with very strange distributions of availabilities over the months??)\n- split the data set into test and train \n- check when the data starts and ends --> if we don't have full months then it won't be valid (e.g. if it starts towards the end of september\n    then it's skewed data because it'll say it was either completely free/completely booked for that month)\n\n\n"

## Data Anaysis


In [ ]:

city = available_all_loc['location'].unique()

NameError: name 'available_all_loc' is not defined